In [ ]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader

import os
import matplotlib.pyplot as plt

In [ ]:
# 하이퍼파라미터 설정
batch_size = 64
epochs = 20
log_interval = 10
learning_rate = 1e-3
rep_dim = 1024
latent_dim = 50  # 잠재 공간의 차원

In [ ]:
# 랜덤 시드 설정
seed = 975
torch.manual_seed(seed)

In [ ]:
# 디바이스 설정
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available() and not use_cuda  # macOS에서 MPS 사용 가능 여부

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [ ]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = DataLoader(datasets.LFWPeople('./data', download=True,transform=transforms.ToTensor()),
                          batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 31 * 31, rep_dim),
            nn.ReLU(),
        )
        
        # z
        self.fc1 = nn.Linear(rep_dim, latent_dim) # mu
        self.fc2 = nn.Linear(rep_dim, latent_dim) # logvar
        self.fc3 = nn.Linear(latent_dim, rep_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(rep_dim, 128 * 31 * 31),
            nn.ReLU(),
            nn.Unflatten(1, (128, 31, 31)),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )
        
    def encode(self, x):
        h = self.encoder(x)
        return self.fc1(h), self.fc2(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def decode(self, z):
        h = self.fc3(z)
        return self.decoder(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [ ]:
# 모델 및 옵티마이저 설정
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


# def test(epoch):
    # model.eval()
    # test_loss = 0
    # with torch.no_grad():
        # for i, (data, _) in enumerate(test_loader):
            # data = data.to(device)
            # recon_batch, mu, logvar = model(data)
            # test_loss += loss_function(recon_batch, data, mu, logvar).item()
            # if i == 0:  # 첫 번째 배치에 대해서만 이미지 시각화
                # n = min(data.size(0), 8)
                # comparison = torch.cat([data[:n], recon_batch.view(-1, 3, 32, 32)[:n]])
                # comparison = comparison.cpu().numpy()
                # fig, axes = plt.subplots(2, n, figsize=(10, 2))
                # for idx in range(n):
                    ## 3채널 이미지를 시각화하기 위해 permute 사용
                    # axes[0, idx].imshow(comparison[idx].transpose(1, 2, 0))
                    # axes[0, idx].axis('off')
                    # axes[1, idx].imshow(comparison[idx + n].transpose(1, 2, 0))
                    # axes[1, idx].axis('off')
                # plt.show()

    # test_loss /= len(test_loader.dataset)
    # print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
if __name__ == "__main__":
    for epoch in range(1, epochs + 1):
        train(epoch)
        # test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, latent_dim).to(device)
            sample = model.decode(sample).cpu()
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.set_xticks([])
            ax.set_yticks([])
            ax.imshow(make_grid(sample).permute(1, 2, 0))
            plt.show()